In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os

# Transfer Model

In [ ]:
pre_model = None
pre_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)
# pre_model = pre_model.cuda()
for param in pre_model.parameters():
  param.requires_grad = False

summary(pre_model, (3,224,224))

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [ ]:
for title, param in pre_model.named_parameters():
  print(title, param.numel())

features.0.weight 1728
features.0.bias 64
features.3.weight 73728
features.3.bias 128
features.6.weight 294912
features.6.bias 256
features.8.weight 589824
features.8.bias 256
features.11.weight 1179648
features.11.bias 512
features.13.weight 2359296
features.13.bias 512
features.16.weight 2359296
features.16.bias 512
features.18.weight 2359296
features.18.bias 512
classifier.0.weight 102760448
classifier.0.bias 4096
classifier.3.weight 16777216
classifier.3.bias 4096
classifier.6.weight 4096000
classifier.6.bias 1000


In [ ]:
for title, param in pre_model.named_parameters():
  if title.startswith('features.18'):
    param.requires_grad = True


In [ ]:
class New_model(nn.Module):

  def __init__(self, pre_model, classes):
    super().__init__()

    self.pre = pre_model
    self.pre.classifier = nn.Sequential(
        nn.Linear(512*7*7, 128),
        nn.Linear(128, classes)
    )

  def forward(self, x):
    x = self.pre(x)
    return x

In [ ]:
if torch.cuda.is_available():
    print("Using GPU")
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
model = New_model(pre_model, 200)
model.to(device)
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [ ]:

for title, param in model.named_parameters():
  if(param.requires_grad):
    print(title, param.numel())

pre.features.18.weight 2359296
pre.features.18.bias 512
pre.classifier.0.weight 3211264
pre.classifier.0.bias 128
pre.classifier.1.weight 25600
pre.classifier.1.bias 200


# DataSet Preparation



In [ ]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

fatal: destination path 'IMagenet' already exists and is not an empty directory.
test  train  val  wnids.txt  words.txt


In [ ]:
classes = open('IMagenet/tiny-imagenet-200/wnids.txt').read().splitlines()
class_id = {classes[i]:i for i in range(len(classes))}
print(class_id)

{'n02124075': 0, 'n04067472': 1, 'n04540053': 2, 'n04099969': 3, 'n07749582': 4, 'n01641577': 5, 'n02802426': 6, 'n09246464': 7, 'n07920052': 8, 'n03970156': 9, 'n03891332': 10, 'n02106662': 11, 'n03201208': 12, 'n02279972': 13, 'n02132136': 14, 'n04146614': 15, 'n07873807': 16, 'n02364673': 17, 'n04507155': 18, 'n03854065': 19, 'n03838899': 20, 'n03733131': 21, 'n01443537': 22, 'n07875152': 23, 'n03544143': 24, 'n09428293': 25, 'n03085013': 26, 'n02437312': 27, 'n07614500': 28, 'n03804744': 29, 'n04265275': 30, 'n02963159': 31, 'n02486410': 32, 'n01944390': 33, 'n09256479': 34, 'n02058221': 35, 'n04275548': 36, 'n02321529': 37, 'n02769748': 38, 'n02099712': 39, 'n07695742': 40, 'n02056570': 41, 'n02281406': 42, 'n01774750': 43, 'n02509815': 44, 'n03983396': 45, 'n07753592': 46, 'n04254777': 47, 'n02233338': 48, 'n04008634': 49, 'n02823428': 50, 'n02236044': 51, 'n03393912': 52, 'n07583066': 53, 'n04074963': 54, 'n01629819': 55, 'n09332890': 56, 'n02481823': 57, 'n03902125': 58, 'n0340

In [ ]:
import os
import glob
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import ToTensor

class TrainDataset(Dataset):
    def __init__(self, root_dir, class_id, transform=None):
        self.root_dir = root_dir
        self.class_id_dict = class_id
        self.transform = transform
        self.class_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
        self.image_paths = []
        self.labels = []

        for class_folder in self.class_folders:
          img_paths = glob.glob(os.path.join(root_dir, class_folder,'images' ,'*.JPEG'))
          self.image_paths.extend(img_paths)
          self.labels.extend([class_folder] * len(img_paths))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.class_id_dict[self.labels[idx]]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

class ValDataset(Dataset):
    def __init__(self, root_dir, class_id, image_class_dict, transform=None):
        self.root_dir = root_dir
        self.class_id_dict = class_id
        self.image_class = image_class_dict
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(root_dir,'images' ,'*.JPEG'))
        self.labels = []

        for img in self.image_paths:
          img_name = img.split('/')[-1]
          self.labels.append(img_name)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.class_id_dict[self.image_class[self.labels[idx]]]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
img_desc = open('IMagenet/tiny-imagenet-200/val/val_annotations.txt').read().splitlines()
image_class = {i.split('\t')[0]: i.split('\t')[1] for i in img_desc}


In [ ]:
transform = transforms.Compose([transforms.Resize(244), transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                               ])
train_dataset = TrainDataset('IMagenet/tiny-imagenet-200/train/', class_id, transform)
val_dataset = ValDataset('IMagenet/tiny-imagenet-200/val/', class_id,image_class, transform)
train_loader = DataLoader(train_dataset, batch_size = 256, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 256, shuffle = True)

# Training

In [ ]:

num_epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
from tqdm import tqdm
train_losses, test_losses = [], []
to_print = True
for e in range(num_epochs):
  running_loss = 0
  for images, labels in tqdm(train_loader, desc ="Training"):

    optimizer.zero_grad()
    labels = labels.to(device)
    images = images.to(device)
    output = model(images)

    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  else:
    test_loss = 0
    accuracy = 0
    with torch.no_grad():
      model.eval()

      # Validation pass
      for images, labels in val_loader:

        labels = labels.to(device)
        images = images.to(device)
        log_ps = model(images)

        test_loss += criterion(log_ps, labels)

        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

    model.train()
    train_losses.append(running_loss/len(train_loader))
    test_losses.append(test_loss/len(val_loader))

    print("Epoch: {}/{}..".format(e+1, num_epochs),
          "Training loss: {:.3f}..".format(running_loss/len(train_loader)),
          "Test loss: {:.3f}..".format(test_loss/len(val_loader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(val_loader)))

Training:  19%|█▊        | 73/391 [2:28:39<10:48:54, 122.44s/it]